# Frac Production Modeling
This is a continuation of Frac Produciton Data Cleaning and Frac Production Analysis notebooks.

Here I will model with the data and optimize the models and features.

In [24]:
# Necessary Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Model Imports
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR

Great, now I will write two functions to streamline training and testing the models.  The training function will fit the model and then cross validate the scores and return the scores.  The test function will cross validate and return the scores.

In [2]:
def train_model(model, X, Y, cv):
    model.fit(X, Y)
    scores = cross_val_score(model, X, Y, cv=cv)
    return 'Training Scores: {:0.4f} (+/- {:0.4f})'.format(scores.mean(), scores.std()*2)

def test_model(model, X, Y, cv):
    scores = cross_val_score(model, X, Y, cv=cv)
    return 'Test Scores: {:0.4f} (+/- {:0.4f})'.format(scores.mean(), scores.std()*2)


In [3]:
# Reading in ALL Colorado Features (pre-feature selection)
co_all = pd.read_csv('fracwells_co.csv', index_col=0)
co_all.head()

,WellName,WellNumber,OperatorAlias,Field,TargetFormation,TotalDepth,VerticalDepth,DrillType,UpperPerforation,LowerPerforation,...,CumOil,First6BOE,FirstProdDate,Township,Range,slick,gel,xlinkgel,hybrid,sandmass
5001097530000,WEP,4-28-11-3-64,AXIS ENERGY,DJ HORIZONTAL NIOBRARA,NIOBRARA,11431.00,7551.00,H,7519.0,11431.0,...,21072.0,10662.0,2017-04-01,03S,64W,0,0,0,0.0,0.000000e+00
5001097540000,WEP,1-28-11-3-64,AXIS ENERGY,DJ HORIZONTAL NIOBRARA,NIOBRARA,11715.00,7594.00,H,8151.0,11715.0,...,23517.0,13185.0,2013-02-01,03S,64W,0,0,0,0.0,0.000000e+00
5001098010000,SHARP,24-3-11HC,WARD PETR,WATTENBERG,CODELL,10161.00,7774.00,H,NaN,NaN,...,142659.0,46241.0,2015-01-01,01S,67W,1,0,1,1.0,2.533881e+06
5001097850000,STATE OF CO,1S-66-36-1609CH,EXTRACTION O&G,THIRD CREEK,NIOBRARA,12177.31,7576.73,H,NaN,NaN,...,16285.0,7094.0,2014-06-01,01S,66W,1,0,1,1.0,3.274332e+06
5001097830000,STATE OF CO,1S-66-36-0108BH,EXTRACTION O&G,THIRD CREEK,NIOBRARA,12164.98,7511.68,H,NaN,NaN,...,16458.0,8304.0,2014-06-01,01S,66W,1,0,1,1.0,3.045143e+06


## Modeling with ALL Colorado Features

In [4]:
# Dropping well identifiers
co_all = co_all.drop(['OperatorAlias','WellName','WellNumber','Township','Range','Field',
                      'FirstProdDate','CumBOE','CumGas','CumOil'], axis=1)

co_all = co_all.dropna(axis=0)
co_all_d = pd.get_dummies(co_all)

co_all_features = co_all_d.drop('First6BOE', axis=1)
co_all_target = co_all_d['First6BOE']

Xa_train, Xa_test, ya_train, ya_test = train_test_split(co_all_features, co_all_target, test_size=0.25, random_state=42)

### Random Forest Regression

In [5]:
# Instantiate the model
rfra = RandomForestRegressor()
# Fit the model and generate training scores
rfra_train = train_model(rfra, Xa_train, ya_train, 5)
# Generate test scores
rfra_test = test_model(rfra, Xa_test, ya_test, 5)
print(rfra_train)
print(rfra_test)

Training Scores: 0.5347 (+/- 0.0624)
Test Scores: 0.5053 (+/- 0.0914)


In [6]:
from sklearn.feature_selection import RFE
rfr = RandomForestRegressor()
selector = RFE(rfr)
selector = selector.fit(co_all_features, co_all_target)

sel_features = pd.DataFrame(selector.support_,
                                   index = co_all_features.columns,
                                    columns=['Selected']).sort_values('Selected', ascending=False)
sel_features

,Selected
TotalDepth,True
gel,True
TargetFormation_NIOBRARA,True
VerticalDepth,True
sandmass,True
hybrid,True
xlinkgel,True
TargetFormation_CODELL,True
slick,True
HorizontalLength,True


From this recursive feature elimination, I will need to eliminate the Target Formation feature and the Drill Type feature, as they are the least important in this model.

In [7]:
co_all = co_all.drop(['DrillType','TargetFormation'], axis=1)

Now, let's get a final baseline of these features for modeling using the random forest regressor.  I need to drop features and then split again.

In [8]:
co_all = pd.get_dummies(co_all)

co_sel_features = co_all.drop('First6BOE', axis=1)
co_all_target = co_all['First6BOE']

Xs_train, Xs_test, ys_train, ys_test = train_test_split(co_sel_features, co_all_target, test_size=0.25, random_state=42)

Now, let's check how these selected features did with the random forest model.

In [9]:
# Instantiate the model
rfrs = RandomForestRegressor()
# Fit the model and generate training scores
rfrs_train = train_model(rfrs, Xs_train, ys_train, 5)
# Generate test scores
rfrs_test = test_model(rfrs, Xs_test, ys_test, 5)
print(rfrs_train)
print(rfrs_test)

Training Scores: 0.5431 (+/- 0.0805)
Test Scores: 0.4850 (+/- 0.0722)


In [10]:
feature_importances = pd.DataFrame(rfrs.feature_importances_,
                                   index = Xs_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)
feature_importances

,importance
TotalDepth,0.298080
LowerPerforation,0.147924
GrossPerforatedInterval,0.141103
VerticalDepth,0.122703
HorizontalLength,0.091014
UpperPerforation,0.089404
sandmass,0.072732
TreatmentJobCount,0.010887
hybrid,0.009126
slick,0.006977


Alright, this is our baseline model now.  An R-squared value of 0.4793 (+/- 0.17).

## Modeling with Selected and Engineered Features
Now I will try with the engineered features to see how they performed.

In [11]:
# Reading in Colorado Features
cofeatures = pd.read_csv('cofeatures', index_col=0)
cofeatures.head()

,gel,slick,xlinkgel,VerticalDepth,HorizontalLength,GrossPerforatedInterval,nphf_sqrt,sandmass,sqrtsandmass,location,First6BOE,TargetFormation_CODELL,TargetFormation_NIOBRARA,TargetFormation_OTHER,hzlen_bin_1-2,hzlen_bin_<1,hzlen_bin_>2,County_Adams,County_Larimer,County_Weld
5001098010000,0.0,1.0,1.0,7774.00,2247.84,2901.282977,73.120155,2.533881e+06,1591.816874,0,46241.0,1,0,0,0,1,0,1,0,0
5001097820000,0.0,1.0,1.0,7574.72,4525.40,5120.217574,73.519946,2.784211e+06,1668.595385,0,7118.0,0,1,0,0,1,0,1,0,0
5001098410000,0.0,0.0,0.0,8045.00,4463.02,5059.443262,73.509025,3.141628e+06,1772.463745,0,23404.0,1,0,0,0,1,0,1,0,0
5001098450000,0.0,1.0,0.0,7841.00,4658.59,5249.979204,73.543258,3.998957e+06,1999.739152,0,97243.0,1,0,0,0,1,0,1,0,0
5001098470000,0.0,1.0,0.0,7707.00,5786.57,6348.924499,73.740393,3.938537e+06,1984.574673,0,93034.0,1,0,0,0,1,0,1,0,0


Knowing that this data is clean, I will first identify my variables, with the first 6 month's production as the target and the remainder as features in the model.  I will then split both variables into training and test sets.

In [12]:
# Identifying all engineered features
cofeats = cofeatures.drop(['First6BOE', 'sandmass', 'HorizontalLength', 'GrossPerforatedInterval'], axis=1)

target = cofeatures.First6BOE

X_train, X_test, y_train, y_test = train_test_split(cofeats, target, test_size=0.25, random_state=42)

### Multivariate Linear Regression
I will start with multivariate linear regression to see how well the model can predict production.

In [13]:
# Instantiate the model
regr = LinearRegression()
# Fit the model and generate training scores
regr_train = train_model(regr, X_train, y_train, 5)
# Generate test scores
regr_test = test_model(regr, X_test, y_test, 5)
print(regr_train)
print(regr_test)

Training Scores: 0.3989 (+/- 0.0442)
Test Scores: 0.3418 (+/- 0.1858)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


Well, that's not great.  The multivariate linear regression model only explains 34% of the variance of the model.  Let's try some other models to see how they do.

### Random Forest Regression
The next model I will use to predict production is random forest regression. 

In [14]:
# Instantiate the model
rfr = RandomForestRegressor()
# Fit the model and generate training scores
rfr_train = train_model(rfr, X_train, y_train, 5)
# Generate test scores
rfr_test = test_model(rfr, X_test, y_test, 5)
print(rfr_train)
print(rfr_test)

Training Scores: 0.6207 (+/- 0.0781)
Test Scores: 0.5023 (+/- 0.1895)


That's better, except the variance in the scores increased from the linear regression models, which is indicative of some overfitting.  

I will use a GridSearchCV to optimize the parameters of the model.  I will optimize the number of estimators, the max features used, the minimum samples split, and the max depth of the tree all for the engineered features.

In [15]:
# Identifying potential parameters
param_grid = { 
            "n_estimators"      : [10, 100, 1000],
            "max_features"      : ["auto", "sqrt", "log2"],
            "min_samples_split" : [4,6,8],
            "max_depth": [4,6,8,10]
            }
# Instantiating grid search
grid = GridSearchCV(estimator=rfr, param_grid=param_grid, cv=5)
# Fitting model
grid.fit(X_train, y_train)
# Identifying best score and best parameters from the Grid Search
print(grid.best_score_)
best_params = grid.best_params_
print(best_params)

0.6300709313684371
{'max_depth': 10, 'max_features': 'auto', 'min_samples_split': 4, 'n_estimators': 1000}


Now that GridSearchCV has identified the optimal parameters from the set, I will use those parameters to fit a model and see what the training and test set scores look like.

In [23]:
# Instantiate the model
rfr_grid = RandomForestRegressor(**best_params)
# Fit the model and generate training scores
rfr_grid_train = train_model(rfr_grid, X_train, y_train, 5)
# Generate test scores
rfr_grid_test = test_model(rfr_grid, X_test, y_test, 5)
print(rfr_grid_train)
print(rfr_grid_test)

Training Scores: 0.6314 (+/- 0.0567)
Test Scores: 0.5700 (+/- 0.1231)


Even better! The test score mean increased, but there is still a sizable variance in the scores between the folds. 

Let's check out the different feature importances to see what features are influencing the model most.

In [17]:
feature_importances = pd.DataFrame(rfr_grid.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)
feature_importances

,importance
hzlen_bin_>2,0.296012
VerticalDepth,0.252622
nphf_sqrt,0.158945
sqrtsandmass,0.132299
hzlen_bin_<1,0.051386
location,0.038211
xlinkgel,0.034392
hzlen_bin_1-2,0.019741
TargetFormation_NIOBRARA,0.006602
TargetFormation_CODELL,0.005907


## Random Forest Quantile Regressor
Because my features and especially my target are non-normally distributed even after tranformations, I will try a random forest quantile regressor.  In a quantile decision tree, the trees predict the percentile and quantile for the data point.  This means that each sample is searched for a leaf that it falls into and a weight is given to that data point based on the fraction of samples in that same leaf, and the weights are summed and averaged.  The output is then the percentile of the data set.

In [18]:
from skgarden import RandomForestQuantileRegressor
rfqr = RandomForestQuantileRegressor(random_state=42)
rfqr_train = train_model(rfqr, X_train, y_train, 5)
rfqr_test = test_model(rfqr, X_test, y_test, 5)
print(rfqr_train)
print(rfqr_test)

Training Scores: 0.6229 (+/- 0.0971)
Test Scores: 0.4728 (+/- 0.2629)




I will use a GridSearchCV to optimize the parameters of the model.  I will optimize the number of estimators, the max features used, the minimum samples split, and the max depth of the tree all for the engineered features.

In [19]:
# Identifying potential parameters
param_grid = { 
            "n_estimators"      : [10, 100, 1000],
            "max_features"      : ["auto", "sqrt", "log2"],
            "min_samples_split" : [2,4,6,8],
            "max_depth": [4,6,8,10]
            }
# Instantiating grid search
grid = GridSearchCV(estimator=rfqr, param_grid=param_grid, cv=5)
# Fitting model
grid.fit(X_train, y_train)
# Identifying best score and best parameters from the Grid Search
print(grid.best_score_)
best_params = grid.best_params_
print(best_params)

0.6312129468215613
{'max_depth': 10, 'max_features': 'sqrt', 'min_samples_split': 2, 'n_estimators': 1000}


Now that GridSearchCV has identified the optimal parameters from the set, I will use those parameters to fit a model and see what the training and test set scores look like.

In [22]:
# Instantiate the model
rfqr_grid = RandomForestRegressor(**best_params)
# Fit the model and generate training scores
rfqr_grid_train = train_model(rfqr_grid, X_train, y_train, 5)
# Generate test scores
rfqr_grid_test = test_model(rfqr_grid, X_test, y_test, 5)
print(rfqr_grid_train)
print(rfqr_grid_test)

Training Scores: 0.6309 (+/- 0.0580)
Test Scores: 0.5718 (+/- 0.1236)


This looks better! I'm still surprised at how low the overall R-squared score is, but there is a large amount of variablility in this data, not only in the production, but also each of the individual well characteristics, including frac method. 

For now, I will finish analyzing the Colorado data by inspecting the partial dependency plots because the overall goal is to better understand how each feature contributes to the production of the well.  

## Partial Dependency Plots
Partial dependency plots enable a view inside of the black boxes that are random forests.  They show how the outcome depends on each individual feature in the model.  This way we can get "coefficients" for each feature and see anomalies in the patterns.

In [31]:
X_train.head()

,gel,slick,xlinkgel,VerticalDepth,nphf_sqrt,sqrtsandmass,location,TargetFormation_CODELL,TargetFormation_NIOBRARA,TargetFormation_OTHER,hzlen_bin_1-2,hzlen_bin_<1,hzlen_bin_>2,County_Adams,County_Larimer,County_Weld
5123414250000,0.0,1.0,0.0,7186.0,82.492303,2229.388405,0,0,1,0,0,0,1,0,0,1
5123394120000,1.0,1.0,0.0,7432.0,79.435257,1503.337507,6,1,0,0,0,1,0,0,0,1
5123413810000,0.0,1.0,0.0,7146.0,84.647209,2251.246907,0,0,1,0,0,0,1,0,0,1
5123352520000,0.0,1.0,1.0,7392.0,73.523953,1761.094156,0,1,0,0,0,1,0,0,0,1
5123403560000,0.0,1.0,1.0,7138.0,73.579858,2075.496411,0,0,1,0,0,1,0,0,0,1


In [48]:
from sklearn.ensemble.partial_dependence import partial_dependence, plot_partial_dependence

gbr = GradientBoostingRegressor()
gbr.fit(X_train, y_train)

feats = [[[3,4], ['VerticalDepth','nphf_sqrt']],
         [[0,1], ['gel','slick']],
        [[2,12], ['xlinkgel','HzLen >2']],
         [[10,11], ['HzLen 1-2', 'HzLen <1']]]

for feat_set in feats:
    print(feat_set)
    gbr_plots = plot_partial_dependence(gbr,
                                features=feat_set[0], # Column numbers to plot
                                X=X_train, # Raw predictors data
                                n_cols=2,
                                feature_names=feat_set[1], # Labels on graph
                                grid_resolution=100) # Number of values to plot on x-axis

[[3, 4], ['VerticalDepth', 'nphf_sqrt']]


ValueError: All entries of features must be less than len(feature_names) = 2, got 3.

In [49]:
gbr_plots = plot_partial_dependence(gbr,
                            features=[0, 1, 2, 3, 4, 10, 11, 12], # Column numbers to plot
                            X=X_train, # Raw predictors data
                            n_cols=2,
                            feature_names=['gel','slick', 'xlinkgel', 'VerticalDepth','nphf_sqrt',
                                          'HzLen 1-2', 'HzLen <1', 'HzLen >2'], # Labels on graph
                            grid_resolution=100) # Number of values to plot on x-axis

ValueError: All entries of features must be less than len(feature_names) = 8, got 10.